<a href="https://colab.research.google.com/github/tamburins/ESAA_2023/blob/main/ESAA_6_19(201_228).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05 개/고양이 분류
파일형태의 데이터셋을 분석에 활용하기 위해서는 고성능 gpu가 탑재된 워크스테이션이나 피씨에 저장된 데이터셋을 텐서플로 분석환경으로 읽어오는 과정이 필요하다. 하지만 실무에서는 컴퓨터 메모리 또는 그래픽 카드 메모리가 부족한 상황이 발생하기 쉽상이다.

따라서 데이터셋을 배치단위로 나눈 다음 배치 한개를 읽어와 딥러닝 모델에 주입하면 메모리부담없이 학습할 수 있다. 전체 데이터셋을 전부 모델에 입력할 때까지 배치 단위로 읽어오고 주입한느 과정을 반복한다.

이번에는 데이터셋을 배치단위의 여러 부분으로 나누고 반복 객체를 통해 각 배치를 한개씩 모델에 입력하여 훈련하는 방법을 소개한다. 이 과정을 손쉽게 처리하기 위해 텐서플로 케라스는 이미지 데이터 제너레이터 함수를 지원한다.

## 5-1 분석준비
### 5-1-1 데이터셋 다운로드
### 5-1-2 라이브러리 불러오기

In [3]:
import tensorflow as tf
from tensorflowkeras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt


ModuleNotFoundError: ignored

In [4]:
#모델 훈련 및 50에폭 훈련
tc_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train model
tc_history=tc_model.fit(train_aug, validation_data=valid_aug, epochs=20)

NameError: ignored

In [ ]:
# plot loss acc
def plot_loss_acc(history, epoch):

    loss, val_loss = history.history['loss'], history.history['val_loss']
    acc, val_acc = history.history['accuracy'], history.history['val_accuracy']

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    axes[0].plot(range(1, epoch + 1), loss, label='Training')
    axes[0].plot(range(1, epoch + 1), val_loss, label='Validation')
    axes[0].legend(loc='best')
    axes[0].set_title('Loss')

    axes[1].plot(range(1, epoch + 1), acc, label='Training')
    axes[1].plot(range(1, epoch + 1), val_acc, label='Validation')
    axes[1].legend(loc='best')
    axes[1].set_title('Accuracy')

    plt.show()


In [ ]:
plot_loss_acc(tc_history, 50)

훈련 초기부터 검증셋에 대한 정확도가 80을 넘어간다. 즉 사전학습 모델인 resnet을 사용하니 이미지로부터 피처를 빠르게 ㄹ추출하는 것을 확인된다.

### 5-1-3 구글 드라이브 마운트
구글 드라이브 마운트하기

### 5-1-4 압축 파일 해제
파일경로를 source filename에 지정 후 압축 해제한 파일을 저장할 경로를 저장한다.

In [ ]:
drive_path = ''
source_filename=drive_path+''
extract_folder = 'dataset/'
import shutil
shutil.unpack_archive(source_filename, extract_folder)

In [5]:
# 훈련 셋 검증 셋 저장 위치 지정
train_dir = extract_folder+'archive/training_set/training_set'
valid_dir = extract_folder+'archive/test_set/test_set'
print(train_dir)
print(valid_dir)

NameError: ignored

## 5-2 모델 학습
### 5-2-1 imagedatagenerator
imagedatagenerator 함수를 실행하고 rescale 옵션을 지정해 이미지 각 픽셀의 값을 0-1로 정규화하고 객체를 image gen 변수에 할당한다

In [ ]:
image_gen=ImageGenerator(rescale=(1/255.))
image_gen

### 5-2-2 flow from directory 함수
지정 폴더에서 이미지를 가져와 반복 이터레이션이 가능하도록 데이터셋을 처리한다.

훈련셋이 저장되어 잇는 Train dir, batchsize 속성에는 배치를 구성하는 이미지개수 32, 타겟사이즈 속성에는 저장될 이미지의 픽셀사이즈, 클래스 속성에는 클레스 레이블, 클래스 모드에는 이진분류 문제를 나타내는 binary 모드 ,랜덤시드값을 지정한다.

예를 들어 cats 폴더에 들어있는 고양이 이미지를 224, 224, 크기로 리사이징 하고 클래스 레이블은 cats에 해당하는 정수로 레이블 인코딩한다. 이밎를 32장씩 묶어 하나으 ㅣ배치를 구성한다. 검증셋에 대해서도 동일 방식으로 제너레이터 객체를 만든다.

In [ ]:
train_gen = image_gen.flow_from_directory(train_dir, batch_size=32, target_size=(224,224), classes=['cats','dogs'], class_mode='binary', seed=2020)
valid_gen = image_gen.flow_from_directory(valid_dir, batch_size=32, target_size=(224,224), classes=['cats','dogs'], class_mode='binary', seed=2020)


In [6]:
# 하나으 ㅣ배치를 선택해 배치안에 들어있는 32개의 이미지를 정답클래스 레이블과 함께 출력하면 다음과 같다
class_labels=['cats','dogs']
batch=next(train_gen)
images, labels=batch[0], batch[1]

plt.figure(figsize=(16,8))
for i in range(32):
  ax=plt.subplot(4,8,i+1)
     plt.imshow(images[i])
     plt.titles(class_labels[labels[i].astype(np.int)])
     plt.axis('off')
plt.tight_layout()
plt.show()

### 5-2-3 모델 훈련
이진 분류모델을 정의하고 배치정규화 합성곱 풀링으로 구성된 단위브럭을 3개 반복하여 이미지로부터 다양한 피처를 추출하고 최종 분류기로는 덴스 레이어를 사용한다. 최종 출력 레이어는 노드 1개를 갖고 활성화 함수로는 시그모이드를 적용한다.

In [ ]:
def build_model():
  model=tf.keras.Sequential([
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2D(32,(3,3), padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),

      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2D(64,(3,3), padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),

      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Conv2D(128,(3,3), padding='same', activation='relu'),
      tf.keras.layers.MaxPooling2D((2,2)),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dropout(0,5),
      tf.keras.layers.Dense(1, activation='sigmoid'),
  ])
  return model
model=build_model()

옵티마이저 손실함수 지정 후 모델을 컴파일하고 20에폭동안 훈련시킨다. 훈련셋에 대한 정확도는 92, 검증 셋에 대한 정확도는 79로 과대적합이 발생하였다.

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
history=model.fit(train_gen, validation_dsata=valid_gen, epochs=20)

In [ ]:
# plot loss acc

In [ ]:
# 손실함수 그래프에서도 과대적합을 확인 가능
plot_loss_acc(history,20)

### 5-2-4 데이터 증강
image data generator를 사용하면 데티터증강기법을 클래스함수의 매개변수 속성으로 지정할 수 있어 편리하다.

좌우반전의 Horizontal flip, 반시계 방향으로 밀리도록 하는 shear range, 줌으로 확대하는 zoom range 속성을 추가한다.

저장되어있는 폭더에서 이미지를 배치단위로 불러와 데이터 증강기법을 적용하고 목표 크기에 맞춰 미니배치를 구성한다. 이르 모아 반복 이터레이션 객체를 만든다.

모델 인스턴스를 생성하고 훈련한다. 에폭을 40으로 늘리는데 다행히 과대적합이 발생하지 않았다. 에폭을 늘리면 모델 성능ㅇ ㅣ개선될 여지가 남아있다.

In [ ]:
image_gen_aug = ImageDataGenerator(rescale=1/255., horizontal_fliop=True, rotation_range=30, shear_range=0.15, zoom_range=0.3)

train_gen_aug = image_gen_aug.flow_from_directory(train_dir, batch_size=32, target_size=(224,224), classes=['cats', 'dogs'],class_mode='binary', seed=2020)
valid_gen_aug = image_gen_aug.flow_from_directory(valid_dir, batch_size=32, target_size=(224,224), classes=['cats', 'dogs'],class_mode='binary', seed=2020)

In [ ]:
model_aug=build_model()
model_aug.compile(optimizer=tf.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
history_aug=model.fit(train_gen, validation_dsata=valid_gen, epochs=20)

In [ ]:
# 과대적합 과소적합 거의 없이 학습이 잘 진행된다!
plot_loss_auc(history_aug, 20)

# 06 객체 탐지

객체 탐지 혹은 객체 검출은 이미지를 인식하는 컴퓨터 비전 ai 기술이 가장 많이 응용되는 분야로 가장 대표적인 응용 분야로 자율주행차가 대표적인 활용 예시이다. 이는 카메라 또는 센서를 활용하여 도로상황을 파악하여 장애물 사람을 식별하고 다른 자동차의 움직임을 파악한다.

객체 탐지는 입력이미지로부터 여러개의 객체를 찾아내고 각 객체가 무엇을 나내는지 분류하는 두가지 작업을 처리한다. 이미지에서 각 개체의 좌표값을 회귀문제로 접근한다. 위치를 찾아낸 각 객체가 어떤 클래스에 속하는지 분류하는 문제를 다음에 처리한다.

즉 이미지를 인식해 이미지 안에 들어있는 여러 객체의 위치를 찾는 회귀문제와 찾아낸 객체를 분류하는 문제가 결합되어 있다.

## 6-1 텐서플로 허브 활용
텐서플로 허브는 검증된 사전학습 모델을 제공하는 저장ㅎ소로 모델을 배포하여 그댈로 서빙하는 것도 가능하며 전이학습을 거쳐 개별 도메인에 맞게 튜닝 후 배포하는 것도 가능하다.

이번에는 텐서플로 허브에서 제공하는 객체탐지 모델을 사용하여 샘플 이미지로부터 객체를 추출하는 작업을 한다. 먼저 허브 라이브러리를 직접 불러온다.

In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub


### 6-1-1 샘플이미지 준비
이미 학습이 완료된 딥러닝 모델을 사용하므로 별도 모델학습을 진행하지 않는다.
샘플 이미지를 준비하여 진행한다.

In [ ]:

img_path = 'https://upload,wikimedia.org/wikipedia/commons/thumb/c/c4/Gangnam_Seoul_January_2009.jpg/1280px-Gangnam_Seoul_January_2009.jpg'
img = tf.keras.utils.get_file(fname='gangnam', origin=img_path)

img=tf.io.read_file(img)
img=tf.image.decode_jpeg(img, channels=3)
img=tf.image.convert_image_dtype(img, tf.float32)

import matplotlib.pyplot as plt
plt.figure(figsize=(15, 10))
plt.imshow(img)

In [ ]:
# 4차원 텐서로 입력받는 사전학습모델
img_input = tf.expand_dims(img, 0)
img_input.shape

### 6-1-2 사전학습모델

inception resnet v2모델을 사용한다

inception resnet v2 모델의 링크를 클릭하여 선택하면 다음과 같은 화면이 나타나는데 url을 복사하면 모델을 가져올 링크가 복사된다.

앞서 가져온 링크를 Load 함수에 전달해주면 모델을 불러온다. 모델 변수에 저장한다.




In [ ]:
 #TenSOrP1owHub에서모델가져오기- Past erRCNN+InceptionResNetV2
model = tfhub.load('https://tfhub.dev/google/faster_rcnn/openimages_v4/ inception_resnet _v2/1')

In [ ]:
# check signature of model
model.signatures.keys()

In [ ]:
#set default, make instance
obj_detector = model.signatures['default']
obj_detector

### 6-1-3 추론
객체탐지 모델에 앞서 미리 전처리를 통해 준비한 샘플이미지를 입력한다. 모델은 추론을 거쳐 예측값을 반환한다. 딕셔너리 키 배열을 확인하면 다음과 같다

In [ ]:
result=obj_detector(img_input)
result.keys()

이들 중 경계박스좌표, 검출된 클래스 아이디, 검출스코어를 사용하여 검출스코어 개수를 확인한 결과 100개의 객체를 탐지한 것을 알 수 있다.

In [ ]:
len(result['detection_scores'])

In [ ]:
# 객체 탐지 결과를 시각화
boxes = result["detection_boxes"]    # Bounding Box 좌표 예측값
labels = result["detection_class_entities"]   # 클래스 값
scores = result["detection_scores"]   # 신뢰도 (confidence)

# 샘플 이미지 가로 세로 크기
img_height, img_width = img.shape[0], img.shape[1]

# 탐지할 최대 객체의 수
obj_to_detect = 10

# 시각화
plt.figure(figsize=(15, 10))
for i in range(min(obj_to_detect, boxes.shape[0])):
    if scores[i] >= 0.2:
        (ymax, xmin, ymin, xmax) = (boxes[i][0]*img_height, boxes[i][1]*img_width,
                                    boxes[i][2]*img_height, boxes[i][3]*img_width)

        plt.imshow(img)
        plt.plot([xmin, xmax, xmax, xmin, xmin], [ymin, ymin, ymax, ymax, ymin],
                 color='yellow', linewidth=2)

        class_name = labels[i].numpy().decode('utf-8')
        infer_score = int(scores[i].numpy()*100)
        annotation = "{}: {}%".format(class_name, infer_score)
        plt.text(xmin+10, ymax+20, annotation,
                 color='white', backgroundcolor='blue', fontsize=10)



## 6-2 yolo 객체 탐ㅁ지
darknet의 yolo객체 탐지 모델을 사용하여 경계박스오 ㅏ예측 클래스를 서로 다른 문제로 다루지 않고 하나의 회귀문제로 접근하는 개념이다. 즉 하나의 신경망이 한번만 계산해 두가지 잏을 모두 처리한다.

### 6-2-1 darknet yolo 모델 추론
사전 모델 학습을 활용한다.
깃허브 저장소를 코랩 환경으로 다운 후 샘플이미지를 업로드하고 업로드해 파일을 선택하여 업로드한다.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet


gpu 사용을 위해 darknet의 마크 파일 수정 및 다크넷 생성

In [ ]:

# GPU 활성화
%cd darknet
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Darknet 생성
!make


In [ ]:
# 모델 가중치 가져오기
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3


In [ ]:
# 샘플이미지 출력 및 확인
import matplotlib.pylab as plt
import tensorflow as tf

plt.figure(figsize=(15, 10))
img = tf.io.read_file('/content/drive/MyDrive/data/gangnam.jpg')
img = tf.image.decode_jpeg(img, channels=3)
img = tf.image.convert_image_dtype(img, tf.float32)
plt.imshow(img)

In [ ]:
# 다크넷 실행 및 샘플이미지에 대한 객체탐지 ㅜ론
# 객체추출 및 예측확률 계산
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /content/drive/MyDrive/data/gangnam.jpeg


plt.figure(figsize=(15, 10))
img = tf.io.read_file('/content/darknet/predictions.jpg')
img = tf.image.decode_jpeg(img, channels=3)
img = tf.image.convert_image_dtype(img, tf.float32)
plt.imshow(img)

### 6-2-2 나만의 yolo 모델 생성
간단하게 검정 바탕에 도형 3개만 탐지하는 모델을 만들어 모델을 쉽게 이해해보자!
앞의 논문에서는 이미지를 가로 세로 7개의 셀로 나누어 총 49셀을 기본으로 하지만 우리는 가로 세로 3개의 셀로 나누는 방식을 채택한다. 또한 논문에서 한 셀당 2개의 박스를 그리지만 우리는 1개으 ㅣ박스를 그리며 객체종류도3개로 단순화하여 구현한다.

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from google.colab.patches import cv2_imshow


width_size = 256
height_size = 256
channel_size = 3
img_size = (width_size,height_size,channel_size)

# 이미지를 나눌 크기
cell_num = 3

# 찾고자 하는 객체 개수
class_num = 3

# 한셀에 그릴 박스 수
anchor_num = 1
label_num = anchor_num * (5 + class_num)

# 학습 수
epoch_num = 20000

# 로스 비중
loss_p_rate = 1.0
loss_cod_rate = 5.0
loss_c_rate = 1.0
loss_p_no_rate = 0.5

실습을 위해 자료실에서 제공하는 3개의 이미지파일을 사용한다. 업로드 버튼을 이용해 코랩 폴더에 도형파일을 업로드한다.

CV2를 이용하여 랜덤위치에 도형 3개를 그리고 위치를 찾아 경계박스로 나타내며 정답 클래스 레이블까지 반환하는 함수를 정의한다.

In [ ]:
def make_img_label():
    img = np.zeros((height_size + 400, width_size + 400, channel_size))
    label = np.zeros((cell_num, cell_num, label_num))
    num_shape = np.random.randint(1,4)
    i = np.random.choice(range(cell_num), num_shape, replace=False)
    j = np.random.choice(range(cell_num), num_shape, replace=False)

    img_0 = cv2.imread('/content/0.png')
    img_1 = cv2.imread('/content/1.png')
    img_2 = cv2.imread('/content/2.png')

    for n_h in range(num_shape):
        row = i[n_h]
        col = j[n_h]
        shape_type = np.random.randint(0, class_num)
        x_rate = np.random.rand()
        y_rate = np.random.rand()
        w_rate = np.random.rand() * 0.3 + 0.1
        h_rate = np.random.rand() * 0.3 + 0.1

        label[row, col] = [1, x_rate, y_rate, w_rate, h_rate, 0, 0, 0]
        label[row, col, 5 + shape_type] = 1
        x = int(x_rate * width_size / cell_num + col * width_size / cell_num)
        y = int(y_rate * height_size / cell_num + row * height_size / cell_num)
        w = int(w_rate * width_size / 2) * 2
        h = int(h_rate * height_size / 2) * 2
        if(shape_type == 0):
            input_img = cv2.resize(img_0, (w,h))
        if(shape_type == 1):
            input_img = cv2.resize(img_1, (w,h))
        if(shape_type == 2):
            input_img = cv2.resize(img_2, (w,h))
        img[y-int(h/2)+200 : y+int(h/2)+200, x-int(w/2)+200 : x+int(w/2)+200] = input_img
    img = img[200 : 200+height_size, 200 : 200+width_size]

    return img, label

객체 탐지 모델의 성능의 일정수준에 도달하기위해서는 복잡한 구조로 구현되어야 한다. 전이학습 방법을 적용하여 이미지 특징을 추춣하는데 좋은 성능을 갖는 모델을 기본으로 활용하는 것이 좋다.

다음 코드에서는 vgg16 모델을 베이스로 사용하고 conv2d 층과 dense레이어를 마지막 객체 탐지 분류기로 설정해준다. 모델 구조를 요약하여 확인한다.

In [ ]:
vgg_model = tf.keras.applications.VGG16(include_top=False, input_shape=img_size)
vgg_model.trainable=False
i=tf.keras.Input(shape=img_size)
out=tf.keras.layers.Conv2D(256, 3, padding='same')(out)
out=tf.keras.layers.Conv2D(128, 3, padding='same')(out)
out=tf.keras.layers.Conv2D(64, 3, padding='same')(out)
out-tf.keras.layers.Flatten()(out)
out=tf.keras.layers.Dense(1024, activation='relu')(out)
out=tf.keras.layers.Dense(3*3*8, activation='sigmoid')(out)
out=tf.keras.layers.Reshape((3,3,8))(out)
yolo_model = tf.kears.Model(inputs=[i], outputs=[out])
opt=tf.keras.optimizers.Adam(0.00001)

yolo_model.summary()

이미지를 총 9개의 셀로 나누고 셀마다 학습을 진행한다. 객체가 있는 셀의 경우 확률박스위치및크기 클래스 종류 ㅁ모두 학습을 진행하고 객채게 앖는 셀은 객체없는 확률만 학습한다. 각 로스는 미리 정한 비중을 곱하고 전체를 더해 최종 로스를 만들어 모델을 학습한다.



In [ ]:
foc = cv2.VideoWriter_focrcc(*'DIVX')
out = cv2.VideoWriter('hjk_yolo.avi', fcc,1.0(width_size, hight_size))
for e in range(epoch_num):
  img, label=make_img_label()
  img = np.reshape(img.(i, hight_size,width_size, 3))
  label = np.reshape(label, (1,3,3,8))
  lost_p_list=[]
  lost_cod_list=[]
  loss_c_list=[]
  loss_p_no_list=[]
  with tf.GradientTape() as tape:
    pred = yolo_model(img)
    for i in range(3):
      for j in range(3):
        if(label[0.i,j,0]==1):
          loss_p_list.append(tf.square(label[0,i,j,0]- pred[0,i,j,0]))
          loss_cod_list.append(tf.square(label[0,i,j,1]- pred[0,i,j,1]))
          loss_cod_list.append(tf.square(label[0,i,j,2]- pred[0,i,j,2]))
          loss_cod_list.append(tf.square(label[0,i,j,3]- pred[0,i,j,3]))
          loss_cod_list.append(tf.square(label[0,i,j,4]- pred[0,i,j,4]))
          loss_c_list.append(tf.square(label[0,i,j,5]- pred[0,i,j,5]))
          loss_c_list.append(tf.square(label[0,i,j,6]- pred[0,i,j,6]))
          loss_c_list.append(tf.square(label[0,i,j,7]- pred[0,i,j,7]))
        else:
          loss_p_no_list.append(tf.square(label[0,i,j,0]- pred[0,i,j,0]))
    loss_p=tf.keras.reduce_mean(loss_p_list)
    loss_cod = tf.reduce_mean(loss_cod_list)
    loss_c=tf.reduce_mean(loss_c_list)
    loss_p_no = tf.reduce_mean(loss_p_no_list)

    loss=loss_p_rate * loss_p + loss_cod_rate*loss_cod + loss_c_rate*loss_c + loss_p_no_rate*loss_p_no
  vars = yolo_model.trainable_variables
  grad = tape.gradient(loss, vars)
  opt.apply_gradients(zip(grad, vars))

  if(e%100==0):
    img=np.reshape(img, (256, 256, 3))
    label=pred.numpy()
    label=np.reshape(label, (3,3,8))
    sample_img = np.uint8(show_box(img, label))
    out.write(sample_img)
  print(e, '완료',loss.numpy())
out.release()

학습 과정은 동영상 파일로 저장된다. 코랩폴더에서 동영상 파일을 로컬피시로 다운한다. 동영상을 실행해보면 약 만번 학습이 이뤄진 후 객체탐지를 어느정도 수준에서 잘 해는걸 볼 수 있다.